### 作業目的: 熟練以Torchtext進行文本資料讀取

本次作業主要會使用[polarity](http://www.cs.cornell.edu/people/pabo/movie-review-data/)的電影評論來進行使用torchtext資料讀取，學員可以在附件的polarity.tsv看到所使用的資料。

Hint: 這次作業同學可以嘗試使用[torchtext.data.TabularDataset](https://torchtext.readthedocs.io/en/latest/data.html#tabulardataset)，可以更簡易讀取資料

### 載入套件

In [1]:
import torch
import pandas as pd
import numpy as np
from torchtext import data, datasets
import spacy
spacy.load('en')

In [2]:
# 探索資料
# 可以發現資料為文本與類別，而類別即為正評與負評
input_data = pd.read_csv('./polarity.tsv', delimiter='\t', header=None, names=['text', 'label'])
input_data.head()

,text,label
0,films adapted from comic books have had plenty...,1
1,every now and then a movie comes along from a ...,1
2,you've got mail works alot better than it dese...,1
3,jaws is a rare film that grabs your attentio...,1
4,moviemaking is a lot like being the general ma...,1


### 建立Pipeline生成資料

In [3]:
# 建立Field與Dataset
text_field = data.Field(sequential=True, dtype=torch.float64, lower=True, tokenize='spacy')
label_field = data.Field(sequential=False)

In [4]:
# input_data = data.TabularDataset(path='./polarity.tsv', format='tsv',fields=[('text',text_field),('label',label_field)])
# examples =  input_data.examples
# 取的examples並打亂順序
examples = []
for (text, label) in input_data.values:
    examples.append(data.Example.fromlist(data=[text, label],
                                          fields=[('text', text_field),
                                                  ('label', label_field)]))
np.random.shuffle(examples)

# 以8:2的比例切分examples
train_ex = examples[:int(len(examples)*0.8)]
test_ex = examples[int(len(examples)*0.8):]

# 建立training與testing dataset
train_data = data.Dataset(examples=train_ex, fields={'text':text_field, 'label':label_field})
test_data = data.Dataset(examples=test_ex, fields={'text':text_field, 'label':label_field})

train_data[0].label, train_data[0].text

(0,
 [' ',
  'first',
  'rule',
  'of',
  'fight',
  'club',
  'is',
  ',',
  'do',
  "n't",
  'talk',
  'about',
  'fight',
  'club',
  '.',
  'this',
  'quote',
  'deals',
  'directly',
  'with',
  'my',
  'opinion',
  'of',
  'this',
  'film',
  ',',
  'because',
  'quite',
  'frankly',
  ',',
  'it',
  "'s",
  'not',
  'even',
  'worth',
  'mentioning',
  '.director',
  'david',
  'fincher',
  '(',
  'seven',
  ')',
  'has',
  'completely',
  'gone',
  'over',
  'the',
  'edge',
  '.with',
  'crude',
  'humor',
  ',',
  'an',
  'intolerable',
  'script',
  'and',
  '2',
  'hours',
  'and',
  '19',
  'minutes',
  'of',
  'pure',
  'torture',
  ',',
  'i',
  "'m",
  'surprised',
  'i',
  'did',
  "n't",
  'walk',
  'out',
  '.this',
  'just',
  'goes',
  'to',
  'show',
  'that',
  'hype',
  'kills',
  '.it',
  "'s",
  'very',
  'rare',
  'that',
  'a',
  'film',
  'so',
  'hyped',
  'up',
  'as',
  '"',
  'fight',
  'club',
  '"',
  'turns',
  'out',
  'to',
  'be',
  'noteworthy',


In [5]:
# 建立字典
text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

print(f"Vocabularies of index 0-5: {text_field.vocab.itos[:10]} \n")
print(f"words to index {text_field.vocab.stoi}")

Vocabularies of index 0-5: ['<unk>', '<pad>', ',', 'the', 'a', 'and', 'of', 'to', 'is', 'in'] 



words to index defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x000002325511B630>>, {'<unk>': 0, '<pad>': 1, ',': 2, 'the': 3, 'a': 4, 'and': 5, 'of': 6, 'to': 7, 'is': 8, 'in': 9, "'s": 10, '"': 11, 'that': 12, '-': 13, 'it': 14, ')': 15, '(': 16, '.': 17, 'as': 18, 'with': 19, 'for': 20, 'film': 21, 'his': 22, 'this': 23, '.the': 24, 'he': 25, 'are': 26, 'on': 27, 'but': 28, 'i': 29, 'by': 30, "n't": 31, 'be': 32, 'movie': 33, 'who': 34, 'an': 35, 'not': 36, 'one': 37, 'was': 38, 'have': 39, 'from': 40, 'you': 41, 'has': 42, 'at': 43, 'her': 44, 'all': 45, 'they': 46, '?': 47, 'out': 48, 'like': 49, 'about': 50, 'up': 51, 'more': 52, 'so': 53, 'which': 54, 'do': 55, ':': 56, 'or': 57, 'their': 58, 'what': 59, 'does': 60, 'some': 61, 'just': 62, '.it': 63, 'when': 64, 'into': 65, 'there': 66, 'she': 67, 'him': 68, 'than': 69, 'only': 70, 'good': 71, "'": 72, 'time': 73, 'can': 74, 'we': 75, 'would': 76, 'no': 77, 'even': 78, 'its': 79, 'most': 8

In [6]:
# create iterator for training and testing data
train_iter, test_iter = data.Iterator.splits(datasets=(train_data, test_data),
                                             batch_sizes = (3, 3),
                                             repeat=False,  
                                             sort_key = lambda ex:len(ex.text))

In [7]:
for train_batch in train_iter:
    print(train_batch.text, train_batch.text.shape)
    print(train_batch.label, train_batch.label.shape)
    break

tensor([[1.6800e+02, 1.0400e+02, 1.6500e+02],
        [1.1300e+02, 3.0000e+00, 3.0000e+00],
        [9.0000e+00, 1.7220e+03, 1.5170e+03],
        ...,
        [4.4680e+03, 1.0000e+00, 1.0000e+00],
        [5.7750e+03, 1.0000e+00, 1.0000e+00],
        [1.7000e+01, 1.0000e+00, 1.0000e+00]], dtype=torch.float64) torch.Size([895, 3])
tensor([2, 2, 1]) torch.Size([3])
